# Data Preparation Notebook

#### Import Modules

In [1]:
import sys
from pathlib import Path

PROJECT_ROOT = Path.cwd().parent
sys.path.append(str(PROJECT_ROOT))

from src.data_prep import *

#### Load Data

In [2]:
df_raw = load_raw('../data/raw/online_retail_II.xlsx')

#### Scope and Assumptions

Partial Periods
- Before data can be processed, it must be noted that the dataset contains incomplete periods.

In [3]:
date_range(df_raw)

The dataset captures December 01, 2009 through December 09, 2010 .


- This time window limits the effectiveness of time-based analysis. The partial periods are taken into account as analysis proceeds.
- Incomplete periods:
    - 2009
    - 2010
    - December 2010
    - Q4 2009
    - Q4 2010
    - Winter 2010

Returns & Cancellations
- Per the data dictionary, `StockCodes` beginning with "C" are cancellations. 
- It is assumed that all other transactions with a negative `Quantity` are returns
- Sales are defined as a transaction with a positive `Quantity`

#### Dataset Overview

In [4]:
data_info(df_raw)

Dataset Overview
Shape
 Columns:   8
 Rows:      525461
--------------------------------------------------------------------------------

 First 5 rows:


,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


--------------------------------------------------------------------------------

 Data Types and Non-Null Values: 

<class 'pandas.DataFrame'>
RangeIndex: 525461 entries, 0 to 525460
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   Invoice      525461 non-null  object        
 1   StockCode    525461 non-null  object        
 2   Description  522533 non-null  object        
 3   Quantity     525461 non-null  int64         
 4   InvoiceDate  525461 non-null  datetime64[us]
 5   Price        525461 non-null  float64       
 6   Customer ID  417534 non-null  float64       
 7   Country      525461 non-null  str           
dtypes: datetime64[us](1), float64(2), int64(1), object(3), str(1)

--------------------------------------------------------------------------------

 Unique Values:


Invoice        28816
StockCode       4632
Description     4681
Quantity         825
InvoiceDate    25296
Price           1606
Customer ID     4383
Country           40
dtype: int64

--------------------------------------------------------------------------------

 Statistical Summary:


,Quantity,InvoiceDate,Price,Customer ID
count,525461.000000,525461,525461.000000,417534.000000
mean,10.337667,2010-06-28 11:37:36.845018,4.688834,15360.645478
min,-9600.000000,2009-12-01 07:45:00,-53594.360000,12346.000000
25%,1.000000,2010-03-21 12:20:00,1.250000,13983.000000
50%,3.000000,2010-07-06 09:51:00,2.100000,15311.000000
75%,10.000000,2010-10-15 12:45:00,4.210000,16799.000000
max,19152.000000,2010-12-09 20:01:00,25111.090000,18287.000000
std,107.424110,NaN,146.126914,1680.811316


--------------------------------------------------------------------------------

 Missing Values Analysis



,Missing
Invoice,0
StockCode,0
Description,2928
Quantity,0
InvoiceDate,0
Price,0
Customer ID,107927
Country,0


Rows Where Price = £0: 3687


## Data Processing

#### Repair Missings

In [5]:
df_repaired = repair_transactions(df_raw)

Repaired Descriptions:                2563
Repaired Prices:                      3206
Missing customer IDs reassigned to 0: 107927


Null customers have been given a `Customer ID` of 0. These transactions will be excluded from customer analysis.

#### Data Cleaning

Unusual Transactions

- Manual analysis of `StockCode` revealed several transaction types to fall outside of the Sale/Return/Cancellation designation. 

In [6]:
unusual_stock_codes(df_raw)

Unusual StockCodes: ['POST', 'D', 'DOT', 'M', 'C2', 'BANK CHARGES', 'TEST001', 'TEST002', 'PADS', 'm', 'S', 'B', 'ADJUST2', 'AMAZONFEE', 'ADJUST']
Net Value of unusual transactions:        £-52,468.90,    -0.55 % of total.
Gross Value of unusual transactions:      £712,259.44,      6.6 % of total.


- Analysis shows that while unusual transactions represent significant gross transaction value, their net impact is near 0.
- Given this, these transactions can be safely removed.

In [7]:
df_clean = clean_transactions(df_repaired)

Null description rows dropped:        365
Missing price rows dropped:           116
Unusual StockCode rows dropped:       2913


#### Adding Features

In [8]:
df = add_features(df_clean)

Transactions labeled "Sales:"         510730
Transactions labeled "Cancellation:"  9582
Transactions labeled "Return:"        1755
Columns Added:
      ['Type', 'Day', 'Month', 'Weekday', 'Year', 'Time', 'TotalPrice']


#### Checks

Final Row Count

In [9]:
df.shape[0]

522067

Data Sample

In [10]:
df.sample(5)

,Invoice,Type,InvoiceDate,Weekday,Year,Month,Day,Time,CustomerID,Country,StockCode,Description,Quantity,UnitPrice,TotalPrice
3404,489683,Sale,2009-12-02 10:09:00,Wed,2009,12,2,10:09:00,16131,United Kingdom,21238,RED SPOTTY CUP,8,0.85,6.80
205334,509397,Sale,2010-05-23 11:04:00,Sun,2010,5,23,11:04:00,17150,United Kingdom,21524,DOOR MAT SPOTTY HOME SWEET HOME,3,7.49,22.47
345367,523213,Sale,2010-09-21 10:47:00,Tue,2010,9,21,10:47:00,16945,United Kingdom,20713,JUMBO BAG OWLS,20,1.95,39.00
485667,535251,Sale,2010-11-25 13:03:00,Thu,2010,11,25,13:03:00,16570,United Kingdom,22835,HOT WATER BOTTLE I AM SO POORLY,18,4.65,83.70
80916,496882,Sale,2010-02-04 14:13:00,Thu,2010,2,4,14:13:00,15167,United Kingdom,84466,TOP SECRET PEN SET,2,1.25,2.50


Data Types

In [11]:
df.dtypes

Invoice                object
Type                      str
InvoiceDate    datetime64[us]
Weekday                   str
Year                    int32
Month                   int32
Day                     int32
Time                   object
CustomerID              Int64
Country                   str
StockCode              object
Description            object
Quantity                int64
UnitPrice             float64
TotalPrice            float64
dtype: object

`InvoiceDate` sucessfully cast as datetime.

Nulls

In [12]:
print("\n Missing Values Analysis\n")
display(pd.DataFrame(df.isna().sum(), columns=['Missing']))
print('Rows Where Price = $0:', df[df['TotalPrice'] == 0].shape[0])


 Missing Values Analysis



,Missing
Invoice,0
Type,0
InvoiceDate,0
Weekday,0
Year,0
Month,0
Day,0
Time,0
CustomerID,0
Country,0


Rows Where Price = $0: 0


No nulls remain after processing.

#### Data Cleaning Summary
- Repaired 2,563 Missing Descriptions and 3,206 Missing Prices
- Dropped 3,394 invalid rows
- Parsed Date Columns
- Added `TotalPrice` (`Quantity` * `UnitPrice`)
- Labeled Sales/Returns/Cancellations

#### Export Data

In [13]:
df.to_csv('../data/processed/cleaned_transactions.csv', index=False)